In [35]:

import pandas as pd
import os
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO, filename="revenues_by_location_logging.log",filemode="w")

payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv', parse_dates=['payment_date'])
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv', parse_dates=['created_at'])


merged_df=payments.merge(orders,on='payment_id',how='left',indicator=True)

#payment_date_converted=pd.DatetimeIndex(merged_df['payment_date'])

merged_df['Year']=pd.DatetimeIndex(merged_df['payment_date']).year
merged_df['Month']=pd.DatetimeIndex(merged_df['payment_date']).month  

merged_df=merged_df.sort_values(by='payment_date',ascending=True)

merged_df=merged_df.drop_duplicates(subset='payment_id',keep='first')#removing duplicates with same payment_id
columns_todrop=['customer_id_x','emp_id','created_at','payment_type', #removing columns that are not needed
                'order_id','item_id','customer_id_y','delivery',
                'delivery_address_id','item_quantity',
                'customer_modifications','delivery_notes','_merge']

merged_df=merged_df.drop(columns=columns_todrop,axis=1)

columns=['cafe_id','payment_date','Year','Month','payment_id','amount']
merged_df=merged_df[columns]

merged_df['payment_date']=pd.to_datetime(merged_df['payment_date'])
merged_df['Year']=merged_df['Year'].astype('str')
merged_df['Month']=merged_df['Month'].astype('str')
#using .loc to change all values in column 'Month' to 2 digits
merged_df['Month'] = merged_df['Month'].str.zfill(2)
merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

#rearranging columns and ordering by cafe_id and Year_Month
first_column = merged_df.pop('Year_Month') 
merged_df.insert(0, 'Year_Month', first_column)
first_column = merged_df.pop('cafe_id') 
merged_df.insert(0, 'cafe_id', first_column)
merged_df=merged_df.sort_values(by=['cafe_id','Year_Month'],ascending=True)

#calculating total monthly revenue,monthly average check,monthly number of orders in each location 
cafe_locations_monthly_df=merged_df.groupby(['cafe_id','Year_Month','Year','Month']).agg(
    
    what_the_avg_check_was_this_month=('amount','mean'),
    number_of_orders_this_month=('payment_id','count'),
    monthly_revenue_total=('amount','sum')
).reset_index()

cafe_locations_monthly_df['what_the_avg_check_was_this_month']=pd.DataFrame(cafe_locations_monthly_df['what_the_avg_check_was_this_month'].apply(lambda x:round(x,2)))

#calculating  average revenue for each month of the year (Jan-Dec) in each location 
cafe_locations_monthly_avg_df=cafe_locations_monthly_df.groupby(['cafe_id','Month']).agg(
   avg_revenue_month_of_the_year_by_location=('monthly_revenue_total','mean')
).reset_index()
cafe_locations_monthly_avg_df['avg_revenue_month_of_the_year_by_location']=pd.DataFrame(cafe_locations_monthly_avg_df['avg_revenue_month_of_the_year_by_location'].apply(lambda x:round(x,2)))


#calculating  average monthly revenue in each location 
cafe_locations_avg_df=cafe_locations_monthly_df.groupby(['cafe_id']).agg(
   avg_monthly_revenue_by_location=('monthly_revenue_total','mean')
).reset_index()
cafe_locations_avg_df['avg_monthly_revenue_by_location']=pd.DataFrame(cafe_locations_avg_df['avg_monthly_revenue_by_location'].apply(lambda x:round(x,2)))

#joining all 3 dataframes 
cafe_locations_monthly_revenue_df=pd.merge(cafe_locations_monthly_df,cafe_locations_monthly_avg_df,how='left',on=['cafe_id','Month'])
cafe_locations_monthly_revenue_df.reset_index()
cafe_locations_df=pd.merge(cafe_locations_monthly_revenue_df,cafe_locations_avg_df,how='left',on=['cafe_id'])
cafe_locations_df=cafe_locations_df.reset_index(drop=True)

#adding running total by month for each location (starts over in january of each year)
cafe_locations_df['Running_total_by_month']=pd.DataFrame(cafe_locations_df.groupby(['cafe_id','Year'])['monthly_revenue_total'].cumsum(
).reset_index(drop=True))



#adding moving monthly averages by location (average is calculated for previous 3 months)
Moving_average=cafe_locations_df['monthly_revenue_total'].rolling(window=3).mean().apply(lambda x: round(x,2)).reset_index(drop=True)
cafe_locations_df['Moving_average_last_3_month']=Moving_average
cafe_locations_df.head()
cafe_locations_df.loc[:,['cafe_id','Year_Month','Year','Month','what_the_avg_check_was_this_month',
                         'number_of_orders_this_month','monthly_revenue_total',
                         'Running_total_by_month','Moving_average_last_3_month',
                         'avg_revenue_month_of_the_year_by_location','avg_monthly_revenue_by_location']]


#changing cafe_id 1,2,3,4,5 TO "New York","Miami","Lima","Buenos Aires","Seattle"
cafe_id_dictionary ={1: "New York", 2: "Miami", 3: "Lima",4: "Buenos Aires",5: "Seattle"}
cafe_locations_df['cafe_id'] = cafe_locations_df['cafe_id'].map(cafe_id_dictionary)
#adding column "Country"  
country_dictionary={"New York":"USA","Miami":"USA","Lima":"Peru","Buenos Aires":"Argentina","Seattle":"USA"}
cafe_locations_df['Country']=cafe_locations_df['cafe_id'].map(country_dictionary)
#adding column "Region" 
region_dictionary={"USA":"North America","Peru":"LATAM","Argentina":"LATAM"}
cafe_locations_df["Region"]=cafe_locations_df['Country'].map(region_dictionary)

#Creating  files with monthly revenues separately for each of the locations:
cafe_1_df=cafe_locations_df[cafe_locations_df['cafe_id']=="New York"]
cafe_1_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\Monthly_revenues_for_Cafe_1.csv',index=False)
cafe_2_df=cafe_locations_df[cafe_locations_df['cafe_id']=="Miami"]
cafe_2_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\Monthly_revenues_for_Cafe_2.csv',index=False)
cafe_3_df=cafe_locations_df[cafe_locations_df['cafe_id']=="Lima"]
cafe_3_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\Monthly_revenues_for_Cafe_3.csv',index=False)
cafe_4_df=cafe_locations_df[cafe_locations_df['cafe_id']=="Buenos Aires"]
cafe_4_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\Monthly_revenues_for_Cafe_4.csv',index=False)
cafe_5_df=cafe_locations_df[cafe_locations_df['cafe_id']=="Seattle"]
cafe_5_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\Monthly_revenues_for_Cafe_5.csv',index=False)

#Creating csv file for tableau vizualization
monthly_sales_by_venue_df=pd.concat([cafe_1_df,cafe_2_df,cafe_3_df,cafe_4_df,cafe_5_df])
monthly_sales_by_venue_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\monthly_sales_by_venue.csv',index=False)
monthly_sales_by_venue_df.to_csv(r"C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Revenue by cafe location\Monthly_sales_by_venue.csv",index=False)


#exporting monthly revenues for each cafe location to .csv file
cafe_locations_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\Monthly_revenues_by_cafe_location.csv',index=False)

Annual_revenue_df=pd.DataFrame(cafe_locations_df.groupby(['cafe_id','Year'])['monthly_revenue_total'].sum())
Annual_revenue_df=Annual_revenue_df.reset_index()
Annual_revenue_df=Annual_revenue_df.rename(columns={'monthly_revenue_total':'Annual revenue'})


#adding percentage of the total annual revenue by cafe location 
Annual_revenue_df=pd.DataFrame(Annual_revenue_df.loc[:,['Year','cafe_id','Annual revenue']])

Total_annual_revenue_df=pd.DataFrame(Annual_revenue_df.groupby(['Year']).agg(
    Total=('Annual revenue','sum')
)).reset_index()

Annual_revenue_df=pd.merge(Annual_revenue_df,Total_annual_revenue_df,how='left',on='Year')
Annual_revenue_df['Percentage_from_annual_total']=(Annual_revenue_df['Annual revenue']/Annual_revenue_df['Total']*100).apply(lambda x:round(x,2))
Annual_revenue_df=Annual_revenue_df.drop(['Total'],axis=1)

Annual_revenue_df=Annual_revenue_df.sort_values(by='Year')
Annual_revenue_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\Annual_revenues_by_cafe_location.csv',index=False)





In [23]:
cafe_locations_df.head()

,cafe_id,Year_Month,Year,Month,what_the_avg_check_was_this_month,number_of_orders_this_month,monthly_revenue_total,avg_revenue_month_of_the_year_by_location,avg_monthly_revenue_by_location,Running_total_by_month,Moving_average_last_3_month,Country,Region
0,New York,2020-02,2020,02,55.92,1,55.92,1087.07,1320.43,55.92,NaN,USA,North America
1,New York,2020-03,2020,03,34.15,37,1263.48,1166.80,1320.43,1319.40,NaN,USA,North America
2,New York,2020-04,2020,04,37.21,31,1153.64,1343.83,1320.43,2473.04,824.35,USA,North America
3,New York,2020-05,2020,05,28.22,36,1015.82,1469.89,1320.43,3488.86,1144.31,USA,North America
4,New York,2020-06,2020,06,32.57,33,1074.74,1362.42,1320.43,4563.60,1081.40,USA,North America


In [27]:
cafe_1_df.head()

,cafe_id,Year_Month,Year,Month,what_the_avg_check_was_this_month,number_of_orders_this_month,monthly_revenue_total,avg_revenue_month_of_the_year_by_location,avg_monthly_revenue_by_location,Running_total_by_month,Moving_average_last_3_month,Country,Region
0,New York,2020-02,2020,02,55.92,1,55.92,1087.07,1320.43,55.92,NaN,USA,North America
1,New York,2020-03,2020,03,34.15,37,1263.48,1166.80,1320.43,1319.40,NaN,USA,North America
2,New York,2020-04,2020,04,37.21,31,1153.64,1343.83,1320.43,2473.04,824.35,USA,North America
3,New York,2020-05,2020,05,28.22,36,1015.82,1469.89,1320.43,3488.86,1144.31,USA,North America
4,New York,2020-06,2020,06,32.57,33,1074.74,1362.42,1320.43,4563.60,1081.40,USA,North America


In [30]:
monthly_sales_by_venue_df=pd.concat([cafe_1_df,cafe_2_df,cafe_3_df,cafe_4_df,cafe_5_df])
monthly_sales_by_venue_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Revenue by cafe location\monthly_sales_by_venue_df.csv',index=False)
monthly_sales_by_venue_df.to_csv(r"C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Revenue by cafe location\Monthly_sales_by_venue.csv",index=False)
